In [1]:
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
import game2048.agents
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent, getBoardFormExpect, MyRnnAgent
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import time
from torch.autograd import Variable
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [2]:
#参数设置
NUM_EPOCHS = 100
BATCH_SIZE = 64
TIME_STEP = 4
INPUT_SIZE = 4
LR = 0.001
trainfilertoread = '/DATA5_DB8/data/gjliu/game2048/dataset/Train2.csv'

In [4]:
#生成数据集
for i in range(1):
    game = Game(4, score_to_win=1024, random=True)
    display2.display(game)
    agent = ExpectiMaxAgent(game, display=display2)
    agent2 = getBoardFormExpect(game, display=display2)
    agent2.writeBoard()
    print(i)


,,2,
,2,,
,,,
,,,


Loaded expectmax lib for 2048: /DB/rhome/gjliu/ML-EE228/2048-api/game2048/expectimax/bin/2048.so
0


In [3]:
class DealDataset(Dataset):

    def __init__(self, root,  transform=None):
        super().__init__()
        Data0 = pd.read_csv(root).values
        self.board = Data0[:, 0:-1]
        self.direc = Data0[:, -1]
        self.len = len(Data0)
        self.transform = transform
        self.idx = 0

    def __getitem__(self, index):

        board = self.board[index].reshape((4, 4))
        
        board = board[ :, :,np.newaxis]

        board = board/11.0
        direc = self.direc[index]
        if self.transform is not None:
            board = self.transform(board)
            board = board.type(torch.float)#更改过board = board.type(torch.float)
        return board, direc


    def __len__(self):

        return self.len

class DealDataset_2(Dataset):

    def __init__(self, root,  transform=None):
        super().__init__()
        Data0 = pd.read_csv(root).values
        self.board = Data0[:, 0:-1]
        self.direc = Data0[:, -1]
        self.len = len(Data0)
        self.transform = transform
        self.idx = 0

    def __getitem__(self, index):

        board = self.board[index].reshape((4, 4))

        board = self.onehot(board)
        board = board.astype(np.int)

        direc = self.direc[index]
        if self.transform is not None:
            board = self.transform(board)
            board = board.type(torch.float)#更改过board = board.type(torch.float)
            
        board = board[  np.newaxis,:,:,:]
        return board, direc

    def onehot(self,board):
        ret = np.zeros(shape=(4,4,12),dtype=bool)
        for r in range(4):
            for c in range(4):
                ret[c,r,board[r,c]] = 1
        return ret
    def __len__(self):

        return self.len
    
class DealDataset(Dataset):

    def __init__(self, root,  transform=None):
        super().__init__()
        Data0 = pd.read_csv(root).values
        self.board = Data0[:, 0:-1]
        self.direc = Data0[:, -1]
        self.len = len(Data0)
        self.transform = transform
        self.idx = 0

    def __getitem__(self, index):

        #board = self.board[index].reshape((4, 4))
        num = int(index/8)
        typ = int(index%8)
        board = self.board[num].reshape((4, 4))
        
        ###数据增强----------
        board1 = self.rote_90_nishiz(board)
        board2 = self.rote_90_nishiz(board1)
        board3 = self.rote_90_nishiz(board2)
        
        board4 = self.vvv(board)
        board5 = self.rote_90_nishiz(board4)
        board6 = self.rote_90_nishiz(board5)
        board7 = self.rote_90_nishiz(board6)
        
        #-----------------end----------
        

        if typ == 0:
            direc = self.direc[num]

        elif typ == 1:
            direc = (self.direc[num]+1)%4
            board = board1
        elif typ == 2:
            direc = (self.direc[num]+2)%4
            board = board2
        
        elif typ == 3:
            direc = (self.direc[num]+3)%4
            board = board3
        elif typ == 4:
            direc = (self.direc[num]+2*((self.direc[num])%2))%4
            board = board4
        elif typ == 5:
            direc = (self.direc[num]+2*((self.direc[num])%2)+1)%4
            board = board5
        
        elif typ == 6:
            direc = (self.direc[num]+2*((self.direc[num])%2)+2)%4
            board = board6
        elif typ == 7:
            direc = (self.direc[num]+2*((self.direc[num])%2)+3)%4
            board = board7
            

        board = board[:, :, np.newaxis]
        board = board/11.0
        #direc = self.direc[index]
        if self.transform is not None:
            board = self.transform(board)
            board = board.type(torch.float)#更改过board = board.type(torch.float)
        return board, direc


    def __len__(self):
        
        lenn = 8*self.len

        return lenn
    
    def rote_90_nishiz(self,board):
        boardnew=np.zeros((4,4))
        boardnew = boardnew.astype(np.int64)

        for i in range(4):
            for j in range(4):
                boardnew[i,j]=board[j,3-i]
        return boardnew
        
    def vvv(self,board):
        boardnew=np.zeros((4,4))
        boardnew = boardnew.astype(np.int64)
        
        for i in range(4):
            for j in range(4):
                boardnew[i,j]=board[3-i,j]
        return boardnew

In [4]:
# class RNN(nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.my_rnn = nn.LSTM(
#             input_size=4,
#             hidden_size=128,
#             num_layers=4,
#             batch_first=True
#         )
        
#         self.conv1 = nn.Conv2d(
#                 in_channels=12,    # 输入通道数
#                 out_channels=6,  # 输出通道数
#                 kernel_size=3,    # 卷积核的尺寸是(5,5)
#                 stride=1,         # 步长为1
#                 padding=1        # 零填充保持图片宽高不变,padding = (kernel_size - stride) / 2
#         )
        
#         self.conv2 = nn.Conv2d(
#                 in_channels=6,    # 输入通道数
#                 out_channels=3,  # 输出通道数
#                 kernel_size=3,    # 卷积核的尺寸是(5,5)
#                 stride=1,         # 步长为1
#                 padding=1        # 零填充保持图片宽高不变,padding = (kernel_size - stride) / 2
#         )
#         self.conv3 = nn.Conv2d(
#                 in_channels=3,    # 输入通道数
#                 out_channels=1,  # 输出通道数
#                 kernel_size=1,    # 卷积核的尺寸是(5,5)
#                 stride=1,         # 步长为1
#                 padding=0         # 零填充保持图片宽高不变,padding = (kernel_size - stride) / 2
#         )

        

#         self.out = nn.Linear(128, 4)
#         self.out2 = nn.Linear(16, 1)#（16，1）
        
#     def forward(self, x):
        
#         w1 = self.conv1(x)

#         w2 = self.conv2(w1)

#         w3 = self.conv3(w2)
    
#         w=w3[:,0,:,:]
        

#         r_out, (h_n, h_c) = self.my_rnn(w,None)
#         out = self.out(r_out[:, -1 ,:])
    
#         return out
    
class RNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.my_rnn = nn.LSTM(
            input_size=4,
            hidden_size=256,
            num_layers=4,
            batch_first=True
        )
        

        self.out = nn.Linear(256, 4)

        
    def forward(self, x):

        r_out, (h_n, h_c) = self.my_rnn(x,None)
        out = self.out(r_out[:, -1 ,:])
        

        return out

    
# class RNN(nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.my_rnn = nn.LSTM(
#             input_size=4,
#             hidden_size=64,
#             num_layers=4,
#             batch_first=True
#         )
        

#         self.out = nn.Linear(64, 4)

        
#     def forward(self, x):

#         r_out, (h_n, h_c) = self.my_rnn(x,None)
#         out = self.out(r_out[:, -1 ,:])
        

#         return out


In [5]:
trainDataset = DealDataset_2(root=trainfilertoread, transform=transforms.Compose(transforms=[transforms.ToTensor()]))
train_loader = DataLoader(dataset=trainDataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
board,direction=trainDataset[0]
print(board.shape)
trainDataset2 = DealDataset(root=trainfilertoread, transform=transforms.Compose(transforms=[transforms.ToTensor()]))
train_loader2 = DataLoader(dataset=trainDataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
board,direction=trainDataset2[0]
print(board.shape)

torch.Size([1, 12, 4, 4])
torch.Size([1, 4, 4])


In [9]:

board,direction=trainDataset2[0]
print(board.shape)
my_rnn = nn.LSTM(
    input_size=4,
    hidden_size=256,
    num_layers=4,
    batch_first=True
)
out = nn.Linear(256, 4)
r_out, (h_n, h_c) = my_rnn(board,None)
print(r_out.shape)


out = out(r_out[:, -1 ,:])
print(out.shape)


torch.Size([1, 4, 4])
torch.Size([1, 4, 256])
torch.Size([1, 4])


In [240]:
my_rnn1 = nn.LSTM(
    input_size=4,
    hidden_size=64,
    num_layers=4,
    batch_first=True
)

conv1 = nn.Conv2d(
            in_channels=16,    # 输入通道数
            out_channels=1,  # 输出通道数
            kernel_size=1,    # 卷积核的尺寸是(5,5)
            stride=1,         # 步长为1
            padding=0        # 零填充保持图片宽高不变,padding = (kernel_size - stride) / 2
)
board,direction=trainDataset[0]

print(board.shape)
print(board.shape[0])
board = board[0,:,:,:]
print(board.shape)
out = nn.Linear(64, 4)
out2 = nn.Linear(16, 1)
r_out, (h_n, h_c) = my_rnn1(board,None)
outtt = out(r_out[:, -1 ,:])
outtt = outtt.transpose(1,0)
print(outtt.shape)
outtt = out2(outtt)

outtt = outtt.transpose(1,0)

print(outtt.shape)


torch.Size([1, 16, 4, 4])
1
torch.Size([16, 4, 4])
torch.Size([4, 16])
torch.Size([1, 4])


In [151]:
my_rnn1 = nn.LSTM(
            input_size=4,
            hidden_size=64,
            num_layers=4,
            batch_first=True
        )
board,direction=trainDataset[0]
board2 = board[0,:,:]
board2= board2[np.newaxis,:, :]
r_out1, (h_n1, h_c1) = my_rnn1(board2,None)
print(r_out1.shape)
r_out2, (h_n1, h_c1) = my_rnn1(board,None)
print(r_out2.shape)
seout = nn.Linear(64, 4)
out2 = seout(r_out2[:, -1 ,:])
print(out2.shape)
out2 = out2.transpose(1,0)
print(out2.shape)
out = seout(r_out1[:, -1 ,:])
print(out.shape)
seout = nn.Linear(16, 1)
out = seout(out2)
print(out.shape)
out = out.transpose(1,0)
print(out.shape)

torch.Size([1, 4, 64])
torch.Size([16, 4, 64])
torch.Size([16, 4])
torch.Size([4, 16])
torch.Size([1, 4])
torch.Size([4, 1])
torch.Size([1, 4])


In [10]:
class MyRnnAgent(Agent):

    def __init__(self, game, display=None):
        if game.size != 4:
            raise ValueError(
                "`%s` can only work with game of `size` 4." % self.__class__.__name__)
        super().__init__(game, display)
        # from .expectimax import board_to_move
        # self.search_func = board_to_move
        # self.search_func = np.random.randint(0, 4)

        self.model = torch.load('/DB/rhome/gjliu/ML-EE228/2048-api/model/rnn_256_model_24.pkl', map_location='cpu')
        self.model.eval()

    def step(self):

        # tmp = self.game.board.flatten()
        # print(self.game.board.ndim)
        board = np.where(self.game.board == 0, 1, self.game.board)
        # print(board)
        board = np.log2(board)
        # print(board)
        # board = board.reshape((4, 4))
        # print(board)
        board = board[:, :, np.newaxis]
        # print(self.game.board)


        board = board/ 11.0
        # print("*******")
        # print(board)
        trans = transforms.Compose([transforms.ToTensor()])
        board = trans(board)
        # print("&&&&&&&")
        # print(board)
        # print(board)
        # board = torch.unsqueeze(board, dim=0)
        board = board.type(torch.float)
        # print("^^^^^^^")
        # print(board)
        out = self.model(board)

        # direction = torch.max(out, 1)[1]
        # direction = torch.max(out, 1)[1]
        # direction = torch.max(out, 1)[1]
        direction = torch.max(out, 1)[1]
        # sleep(3600)
        return int(direction)

In [55]:
class MyOneHotAgent(Agent):

    def __init__(self, game, display=None):
        if game.size != 4:
            raise ValueError(
                "`%s` can only work with game of `size` 4." % self.__class__.__name__)
        super().__init__(game, display)
        # from .expectimax import board_to_move
        # self.search_func = board_to_move
        # self.search_func = np.random.randint(0, 4)

        self.model = torch.load('/DATA5_DB8/data/gjliu/game2048/model/model_onehot/rnn_onehot_model_final_2.pkl', map_location='cpu')
        self.model.eval()

        
    def onehot(self,board):
        ret = np.zeros(shape=(4,4,12),dtype=bool)
        for r in range(4):
            for c in range(4):
                ret[c,r,board[r,c]] = 1
        return ret
    
    
    def step(self):

        board = np.where(game.board == 0, 1, game.board)
        board = np.log2(board)

        #board =board.reshape((16, ))
        board = board.astype(np.int64)

        board = self.onehot(board)
        board = board.astype(np.int)


        trans = transforms.Compose([transforms.ToTensor()])
        board = trans(board)

        board = board.type(torch.float)
        board = board[np.newaxis,:, :,: ]

        out = self.model(board)


        direction = torch.max(out, 1)[1]
        # sleep(3600)
        return int(direction)

In [30]:
class MyRnnVoteAgent(Agent):

    def __init__(self, game, display=None):
        if game.size != 4:
            raise ValueError(
                "`%s` can only work with game of `size` 4." % self.__class__.__name__)
        super().__init__(game, display)
        # from .expectimax import board_to_move
        # self.search_func = board_to_move
        # self.search_func = np.random.randint(0, 4)

        self.model1 = torch.load('/DB/rhome/gjliu/ML-EE228/help/2048-api-help/model/myRNN3Random05Model50rate_0.pkl', map_location='cpu')
        self.model2 = torch.load('/DB/rhome/gjliu/ML-EE228/help/2048-api-help/model/rnn_model_14.pkl', map_location='cpu')

        self.model1.eval()
        self.model2.eval()


    def step(self):

        board = np.where(self.game.board == 0, 1, self.game.board)
        board = np.log2(board)
        board = board[:, :, np.newaxis]
        board = board/ 11.0
        trans = transforms.Compose([transforms.ToTensor()])
        board = trans(board)
        board = board.type(torch.float)

        out1 = self.model1(board)
        direction1 = torch.max(out1, 1)[1]
        out2 = self.model2(board)
        direction2 = torch.max(out2, 1)[1]

        aaa = np.random.randint(0,10) 
        
        if (aaa <0.9 ):

            direction = direction1
        else:
            direction = direction2
        return int(direction)

In [5]:
display1 = Display()
display2 = IPythonDisplay()

In [17]:
trainDataset = DealDataset(root=trainfilertoread, transform=transforms.Compose(transforms=[transforms.ToTensor()]))
train_loader = DataLoader(dataset=trainDataset, batch_size=  64    , shuffle=True, num_workers=0)

In [22]:
print(trainDataset[0])
board,a=trainDataset[0]
model = torch.load('/DATA5_DB8/data/gjliu/game2048/model/model_rnn_256_dataup/rnn_256_model_24.pkl', map_location='cpu')
import time
tic =  time.time()
for i in range(100):
    aa=model(board)


toc =  time.time()
print((toc-tic)/100)

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0909, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.1818, 0.0000, 0.0000]]]), 3)
0.003320491313934326


In [43]:
model = torch.load('/DB/rhome/gjliu/ML-EE228/help/2048-api-help/model/rnn_model_14.pkl', map_location='cpu')
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LR)
#用于测试-------------------------------------
for iiii in range(1):
    
    for kk in range(25):
        board, direc = trainDataset[kk]


        board = board.view(-1,4,4)
        out = model(board)
        print(board)
        pred = torch.max(out, 1)[1]
        print('方向是',pred)
        train_correct = (pred == direc).sum().item()
        print('Epoch: ', epoch, 
              '| test accuracy: %.4f' % (train_correct/(BATCH_SIZE * 1.0)))

tensor([[[0.0000, 0.0000, 0.0909, 0.0000],
         [0.0000, 0.0909, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])
方向是 tensor([2])
Epoch:  0 | test accuracy: 1.0000
tensor([[[0.0000, 0.0000, 0.0000, 0.0909],
         [0.0000, 0.0909, 0.0000, 0.0909],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])
方向是 tensor([3])
Epoch:  0 | test accuracy: 1.0000
tensor([[[0.1818, 0.0909, 0.0000, 0.1818],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])
方向是 tensor([1])
Epoch:  0 | test accuracy: 1.0000
tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.1818, 0.0909, 0.1818, 0.1818]]])
方向是 tensor([3])
Epoch:  0 | test accuracy: 1.0000
tensor([[[0.1818, 0.0909, 0.1818, 0.1818],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 

In [45]:
model = torch.load('/DB/rhome/gjliu/ML-EE228/help/2048-api-help/model/myRNN3Random05Model50rate_0.pkl', map_location='cpu')
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LR)
#用于测试-------------------------------------
for iiii in range(1):
    
    for kk in range(25):
        board, direc = trainDataset[kk]


        board = board.view(-1,4,4)
        out = model(board)
        print(board)
        pred = torch.max(out, 1)[1]
        print('方向是',pred)
        train_correct = (pred == direc).sum().item()
        print('Epoch: ', epoch, 
              '| test accuracy: %.4f' % (train_correct/(BATCH_SIZE * 1.0)))


tensor([[[0.0000, 0.0000, 0.0909, 0.0000],
         [0.0000, 0.0909, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])
方向是 tensor([2])
Epoch:  0 | test accuracy: 1.0000
tensor([[[0.0000, 0.0000, 0.0000, 0.0909],
         [0.0000, 0.0909, 0.0000, 0.0909],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])
方向是 tensor([3])
Epoch:  0 | test accuracy: 1.0000
tensor([[[0.1818, 0.0909, 0.0000, 0.1818],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])
方向是 tensor([1])
Epoch:  0 | test accuracy: 1.0000
tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.1818, 0.0909, 0.1818, 0.1818]]])
方向是 tensor([3])
Epoch:  0 | test accuracy: 1.0000
tensor([[[0.1818, 0.0909, 0.1818, 0.1818],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 

In [220]:
#用于测试
trainDataset = DealDataset(root=trainfilertoread, transform=transforms.Compose(transforms=[transforms.ToTensor()]))
train_loader = DataLoader(dataset=trainDataset, batch_size=  64   , shuffle=True, num_workers=0)
model = RNN2()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LR)
for epoch in range(1):
    for index, (board, direc) in enumerate(train_loader):
        board, direc = Variable(board), Variable(direc)
        if torch.cuda.is_available():
            board, direc = board.cuda(), direc.cuda()
            model.cuda()
        board1,yy=trainDataset[0]
        print(board1.shape)
        print(board.shape)
        board = board.view(-1,4,4)
        print(board.shape)
#         my_rnn = nn.LSTM(
#             input_size=4,
#             hidden_size=256,
#             num_layers=4,
#             batch_first=True
#         )
#         r_out, (h_n, h_c) = my_rnn(board,None)
#         print(r_out[:, -1 ,:].shape)
#         outtt = out(r_out[:, -1 ,:])
        
#         print(r_out.shape)
        outsp = model(board)
        print(outsp.shape)
        loss = criterion(outsp, direc)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        break


torch.Size([1, 4, 4])
torch.Size([64, 1, 4, 4])
torch.Size([64, 4, 4])
torch.Size([64, 4])


In [264]:
#用于测试
trainDataset = DealDataset_2(root=trainfilertoread, transform=transforms.Compose(transforms=[transforms.ToTensor()]))
train_loader = DataLoader(dataset=trainDataset, batch_size=  64   , shuffle=True, num_workers=0)

model = RNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LR)
for epoch in range(1):
    for board, direc in train_loader:
        board, direc = Variable(board), Variable(direc)
#         if torch.cuda.is_available():
#             board, direc = board.cuda(), direc.cuda()
#             model.cuda()
     
        print(board.shape)
        board = board.view(-1,16,4,4)
        print(board.shape)
        out = model(board)
        print(out.shape)
        
        
#         loss = criterion(out, direc)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
        break



torch.Size([64, 1, 16, 4, 4])
torch.Size([64, 16, 4, 4])
torch.Size([64, 4])


In [226]:
#model = RNN()
model = torch.load('/DATA5_DB8/data/gjliu/game2048/model/model_rnn_256_dataup/rnn_256_model_24.pkl', map_location='cpu')

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LR)

In [50]:
for epoch in range(20):
    
    for index, (board, direc) in enumerate(train_loader):
        board, direc = Variable(board), Variable(direc)
        if torch.cuda.is_available():
            board, direc = board.cuda(), direc.cuda()
            model.cuda()
            
        board = board.view(-1,12,4,4)
        out = model(board)
#         print(board.shape)
#         print(direc.shape)
        

        #board = board.view(-1,4,4)
        #out = model(board)
        loss = criterion(out, direc)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # train_loss += loss.data[0]
        # pred = torch.max(out, 1)[1]
        # train_correct = (pred == direc).sum().item()
        # train_acc += train_correct

        if index % 50 == 0:
            out = model(board)
            pred = torch.max(out, 1)[1]
            train_correct = (pred == direc).sum().item()
            print('Epoch: ', epoch, '| train loss: %.4f' % loss,
                  '| test accuracy: %.4f' % (train_correct/(BATCH_SIZE * 1.0)))
    torch.save(model, '/DATA5_DB8/data/gjliu/game2048/model/model_onehot/rnn_onehot_model_' + str(epoch) + '.pkl')
torch.save(model, '/DATA5_DB8/data/gjliu/game2048/model/model_onehot/rnn_onehot_model_final_2.pkl')

Epoch:  0 | train loss: 1.3929 | test accuracy: 0.2344
Epoch:  0 | train loss: 1.3870 | test accuracy: 0.2344
Epoch:  0 | train loss: 1.2577 | test accuracy: 0.4688
Epoch:  0 | train loss: 1.1865 | test accuracy: 0.4844
Epoch:  0 | train loss: 1.2249 | test accuracy: 0.3281
Epoch:  0 | train loss: 1.2461 | test accuracy: 0.3750
Epoch:  0 | train loss: 1.1074 | test accuracy: 0.4375
Epoch:  0 | train loss: 1.1713 | test accuracy: 0.4375
Epoch:  0 | train loss: 1.1005 | test accuracy: 0.4375
Epoch:  0 | train loss: 1.0444 | test accuracy: 0.5469
Epoch:  0 | train loss: 1.0495 | test accuracy: 0.4219
Epoch:  0 | train loss: 1.1862 | test accuracy: 0.3438
Epoch:  0 | train loss: 1.0586 | test accuracy: 0.4688
Epoch:  0 | train loss: 1.0843 | test accuracy: 0.4219
Epoch:  0 | train loss: 1.1405 | test accuracy: 0.4219
Epoch:  0 | train loss: 1.0396 | test accuracy: 0.3906
Epoch:  0 | train loss: 1.1564 | test accuracy: 0.3906
Epoch:  0 | train loss: 1.1464 | test accuracy: 0.3281
Epoch:  0 

Epoch:  0 | train loss: 1.0872 | test accuracy: 0.4844
Epoch:  0 | train loss: 1.1523 | test accuracy: 0.4062
Epoch:  0 | train loss: 1.2672 | test accuracy: 0.3594
Epoch:  0 | train loss: 1.1426 | test accuracy: 0.3906
Epoch:  0 | train loss: 1.0868 | test accuracy: 0.5000
Epoch:  0 | train loss: 1.0878 | test accuracy: 0.4375
Epoch:  0 | train loss: 1.1519 | test accuracy: 0.4688
Epoch:  0 | train loss: 1.0853 | test accuracy: 0.4375
Epoch:  0 | train loss: 1.2202 | test accuracy: 0.3438
Epoch:  0 | train loss: 1.1663 | test accuracy: 0.3125
Epoch:  0 | train loss: 1.2449 | test accuracy: 0.3125
Epoch:  0 | train loss: 1.1709 | test accuracy: 0.2969
Epoch:  0 | train loss: 1.1907 | test accuracy: 0.4062
Epoch:  0 | train loss: 1.1192 | test accuracy: 0.4688
Epoch:  0 | train loss: 1.2024 | test accuracy: 0.4219
Epoch:  0 | train loss: 1.1203 | test accuracy: 0.4688
Epoch:  0 | train loss: 1.0670 | test accuracy: 0.3906
Epoch:  0 | train loss: 1.1723 | test accuracy: 0.3125
Epoch:  0 

Epoch:  0 | train loss: 1.1465 | test accuracy: 0.4219
Epoch:  0 | train loss: 1.1033 | test accuracy: 0.4062
Epoch:  0 | train loss: 1.0903 | test accuracy: 0.4219
Epoch:  0 | train loss: 1.0895 | test accuracy: 0.4219
Epoch:  0 | train loss: 1.2235 | test accuracy: 0.3906
Epoch:  0 | train loss: 1.2177 | test accuracy: 0.3906
Epoch:  0 | train loss: 1.0385 | test accuracy: 0.4688
Epoch:  0 | train loss: 1.0462 | test accuracy: 0.5000
Epoch:  0 | train loss: 1.1199 | test accuracy: 0.3438
Epoch:  0 | train loss: 1.0922 | test accuracy: 0.3906
Epoch:  0 | train loss: 1.1662 | test accuracy: 0.4531
Epoch:  0 | train loss: 1.1497 | test accuracy: 0.4375
Epoch:  0 | train loss: 1.1884 | test accuracy: 0.3906
Epoch:  0 | train loss: 1.0149 | test accuracy: 0.5625
Epoch:  0 | train loss: 1.0477 | test accuracy: 0.5312
Epoch:  0 | train loss: 1.0803 | test accuracy: 0.4375
Epoch:  0 | train loss: 1.1225 | test accuracy: 0.4219
Epoch:  0 | train loss: 1.0057 | test accuracy: 0.5312
Epoch:  0 

Epoch:  0 | train loss: 1.0502 | test accuracy: 0.5156
Epoch:  0 | train loss: 1.0237 | test accuracy: 0.5156
Epoch:  0 | train loss: 1.1273 | test accuracy: 0.4688
Epoch:  0 | train loss: 1.0983 | test accuracy: 0.3438
Epoch:  0 | train loss: 1.1400 | test accuracy: 0.3438
Epoch:  0 | train loss: 1.1467 | test accuracy: 0.4531
Epoch:  0 | train loss: 1.0316 | test accuracy: 0.4844
Epoch:  0 | train loss: 1.1608 | test accuracy: 0.4688
Epoch:  0 | train loss: 0.9900 | test accuracy: 0.5312
Epoch:  0 | train loss: 1.0344 | test accuracy: 0.5000
Epoch:  0 | train loss: 1.0165 | test accuracy: 0.5156
Epoch:  0 | train loss: 1.0043 | test accuracy: 0.5312
Epoch:  0 | train loss: 1.0384 | test accuracy: 0.5156
Epoch:  0 | train loss: 1.1161 | test accuracy: 0.4531
Epoch:  0 | train loss: 0.9576 | test accuracy: 0.6250
Epoch:  0 | train loss: 1.1397 | test accuracy: 0.4688
Epoch:  0 | train loss: 1.0344 | test accuracy: 0.4531
Epoch:  0 | train loss: 1.0864 | test accuracy: 0.4375
Epoch:  0 

Epoch:  0 | train loss: 1.1078 | test accuracy: 0.4219
Epoch:  0 | train loss: 1.1535 | test accuracy: 0.3438
Epoch:  0 | train loss: 1.1248 | test accuracy: 0.5000
Epoch:  0 | train loss: 0.9903 | test accuracy: 0.5000
Epoch:  0 | train loss: 1.1454 | test accuracy: 0.4062
Epoch:  0 | train loss: 0.9908 | test accuracy: 0.5156
Epoch:  0 | train loss: 1.1356 | test accuracy: 0.4844
Epoch:  0 | train loss: 0.9638 | test accuracy: 0.5625
Epoch:  0 | train loss: 1.1320 | test accuracy: 0.4531
Epoch:  0 | train loss: 1.0368 | test accuracy: 0.5625
Epoch:  0 | train loss: 1.1756 | test accuracy: 0.4688
Epoch:  0 | train loss: 1.0115 | test accuracy: 0.4844
Epoch:  0 | train loss: 0.9188 | test accuracy: 0.5312
Epoch:  0 | train loss: 1.0753 | test accuracy: 0.4688
Epoch:  0 | train loss: 1.0132 | test accuracy: 0.4531
Epoch:  0 | train loss: 0.9856 | test accuracy: 0.5625
Epoch:  0 | train loss: 0.9589 | test accuracy: 0.5312
Epoch:  0 | train loss: 1.0945 | test accuracy: 0.4375
Epoch:  0 

...

Epoch:  19 | train loss: 0.6298 | test accuracy: 0.7500
Epoch:  19 | train loss: 0.7307 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.6827 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.7283 | test accuracy: 0.6094
Epoch:  19 | train loss: 0.7763 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.6642 | test accuracy: 0.6562
Epoch:  19 | train loss: 0.7855 | test accuracy: 0.7188
Epoch:  19 | train loss: 0.7466 | test accuracy: 0.7031
Epoch:  19 | train loss: 0.8558 | test accuracy: 0.5938
Epoch:  19 | train loss: 0.6381 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.7302 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.6923 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.7294 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.7078 | test accuracy: 0.7031
Epoch:  19 | train loss: 0.8499 | test accuracy: 0.6562
Epoch:  19 | train loss: 0.7628 | test accuracy: 0.6250
Epoch:  19 | train loss: 0.6773 | test accuracy: 0.7500
Epoch:  19 | train loss: 0.6939 | test accuracy:

Epoch:  19 | train loss: 0.6632 | test accuracy: 0.7188
Epoch:  19 | train loss: 0.7347 | test accuracy: 0.7031
Epoch:  19 | train loss: 0.7921 | test accuracy: 0.6562
Epoch:  19 | train loss: 0.6799 | test accuracy: 0.7188
Epoch:  19 | train loss: 0.7662 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.7352 | test accuracy: 0.7188
Epoch:  19 | train loss: 0.7195 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.7137 | test accuracy: 0.6406
Epoch:  19 | train loss: 0.8269 | test accuracy: 0.6094
Epoch:  19 | train loss: 0.7439 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.8282 | test accuracy: 0.5625
Epoch:  19 | train loss: 0.6562 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.6779 | test accuracy: 0.7344
Epoch:  19 | train loss: 0.7996 | test accuracy: 0.6250
Epoch:  19 | train loss: 0.7402 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.5790 | test accuracy: 0.7344
Epoch:  19 | train loss: 0.7009 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.7374 | test accuracy:

Epoch:  19 | train loss: 0.7249 | test accuracy: 0.6562
Epoch:  19 | train loss: 0.7479 | test accuracy: 0.6094
Epoch:  19 | train loss: 0.6048 | test accuracy: 0.7344
Epoch:  19 | train loss: 0.7921 | test accuracy: 0.6094
Epoch:  19 | train loss: 0.7809 | test accuracy: 0.6250
Epoch:  19 | train loss: 0.7740 | test accuracy: 0.6250
Epoch:  19 | train loss: 0.6643 | test accuracy: 0.7188
Epoch:  19 | train loss: 0.6827 | test accuracy: 0.6562
Epoch:  19 | train loss: 0.7005 | test accuracy: 0.6406
Epoch:  19 | train loss: 0.7557 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.6752 | test accuracy: 0.6094
Epoch:  19 | train loss: 0.8001 | test accuracy: 0.5781
Epoch:  19 | train loss: 0.8463 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.6517 | test accuracy: 0.7188
Epoch:  19 | train loss: 0.8234 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.7706 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.5796 | test accuracy: 0.7500
Epoch:  19 | train loss: 0.9592 | test accuracy:

Epoch:  19 | train loss: 0.5779 | test accuracy: 0.7969
Epoch:  19 | train loss: 0.7451 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.6861 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.5073 | test accuracy: 0.7969
Epoch:  19 | train loss: 0.6732 | test accuracy: 0.7031
Epoch:  19 | train loss: 0.7736 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.7352 | test accuracy: 0.7188
Epoch:  19 | train loss: 0.5822 | test accuracy: 0.7188
Epoch:  19 | train loss: 0.7510 | test accuracy: 0.6562
Epoch:  19 | train loss: 0.6300 | test accuracy: 0.7188
Epoch:  19 | train loss: 0.8808 | test accuracy: 0.5156
Epoch:  19 | train loss: 0.8145 | test accuracy: 0.6406
Epoch:  19 | train loss: 0.7405 | test accuracy: 0.6406
Epoch:  19 | train loss: 0.7313 | test accuracy: 0.6406
Epoch:  19 | train loss: 0.7723 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.6812 | test accuracy: 0.7031
Epoch:  19 | train loss: 0.7383 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.8730 | test accuracy:

Epoch:  19 | train loss: 0.6178 | test accuracy: 0.7812
Epoch:  19 | train loss: 0.6506 | test accuracy: 0.7344
Epoch:  19 | train loss: 0.6572 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.7972 | test accuracy: 0.6094
Epoch:  19 | train loss: 0.7640 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.7203 | test accuracy: 0.6250
Epoch:  19 | train loss: 0.7227 | test accuracy: 0.6406
Epoch:  19 | train loss: 0.7247 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.7769 | test accuracy: 0.6562
Epoch:  19 | train loss: 0.9720 | test accuracy: 0.5156
Epoch:  19 | train loss: 0.7119 | test accuracy: 0.7031
Epoch:  19 | train loss: 0.8002 | test accuracy: 0.6250
Epoch:  19 | train loss: 0.7649 | test accuracy: 0.6562
Epoch:  19 | train loss: 0.7032 | test accuracy: 0.6719
Epoch:  19 | train loss: 0.7066 | test accuracy: 0.7969
Epoch:  19 | train loss: 0.6458 | test accuracy: 0.6875
Epoch:  19 | train loss: 0.5618 | test accuracy: 0.7188
Epoch:  19 | train loss: 0.5930 | test accuracy:

In [11]:
game = Game(4, score_to_win=2048*4, random=False)
game2 =game
display2.display(game)
# my agent
#agent = ExpectiMaxAgent(game, display=display2)

agent = MyRnnAgent(game, display=display2)
#agent = MyOneHotAgent(game, display=display2)
agent.play(verbose=True)

,4,,
,,,
,,,
,,2,


AttributeError: Can't get attribute 'RNN' on <module '__main__'>

In [140]:
agent2 = MyRnnAgent2(game2, display=display2)
agent2.play(verbose=True)

NameError: name 'MyRnnAgent2' is not defined

In [75]:
print(game2.__board)

AttributeError: 'Game' object has no attribute '__board'

In [30]:
game = Game(4, score_to_win=2048, random=False)
display2.display(game)
agent = ExpectiMaxAgent(game, display=display2)
agent.play(verbose=True)

tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0909, 0.0000],
         [0.0000, 0.1818, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]]) 3


In [8]:
Data0 = pd.read_csv('/DB/rhome/gjliu/ML-EE228/2048-api/Train.csv').values

In [9]:
print(np.shape(Data0))
print(type(Data0))
print((Data0[0, 0:-1]))
print((Data0[0, :]))
print(Data0[0, -1])

(696, 17)
<class 'numpy.ndarray'>
[0 0 0 0 0 0 1 0 0 2 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 2 0 0 0 0 0 0 3]
3


In [26]:
board = Data0[:, 0:-1]
board = board[0].reshape((4, 4))
print(board)
print(board.shape)
board = board[:, :, np.newaxis]
board = board/11.0
#print(board)
print(board.shape)
direc = Data0[:, -1]
print(direc[7])
board = transforms.Compose(transforms=[transforms.ToTensor()])(board)
print(board)
print(type(board))
print(board.shape)
board = board.type(torch.float)
print(board)
print(type(board))
print(board.shape)
print(len(Data0))

[[0 0 0 0]
 [0 0 1 0]
 [0 2 0 0]
 [0 0 0 0]]
(4, 4)
(4, 4, 1)
3
tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0909, 0.0000],
         [0.0000, 0.1818, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]], dtype=torch.float64)
<class 'torch.Tensor'>
torch.Size([1, 4, 4])
tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0909, 0.0000],
         [0.0000, 0.1818, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]])
<class 'torch.Tensor'>
torch.Size([1, 4, 4])
696


In [11]:
game = Game(4, score_to_win=2048, random=False)
display2.display(game)
print(game.board)
board = np.where(game.board == 0, 1, game.board)
print(board)
board = np.log2(board)
# print(board)
# board = board.reshape((4, 4))
print(board)
board = board[:, :, np.newaxis]
print(board)

,,,
,2,,
4,,,
,,,


[[0. 0. 0. 0.]
 [0. 2. 0. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[1. 1. 1. 1.]
 [1. 2. 1. 1.]
 [4. 1. 1. 1.]
 [1. 1. 1. 1.]]
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[[0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [1.]
  [0.]
  [0.]]

 [[2.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]]]


In [64]:
model = torch.load('/DB/rhome/gjliu/ML-EE228/2048-api/model/rnn_model_final.pkl', map_location='cpu')
game = Game(4, score_to_win=2048, random=True)
board = np.where(game.board == 0, 1, game.board)
board = np.log2(board)
board = board[:, :, np.newaxis]

board = board/ 11.0

trans = transforms.Compose([transforms.ToTensor()])
board = trans(board)

board = board.type(torch.float)

out = model(board)

direction = torch.max(out, 1)[1]
print(direction)

tensor([1])


In [36]:
def onehot(board):
    ret = np.zeros(shape=(4,4,16),dtype=bool)
    for r in range(4):
        for c in range(4):
            ret[r,c,board[r,c]] = 1
    return ret


Data0 = pd.read_csv('/DATA5_DB8/data/gjliu/game2048/dataset/Train_special.csv').values
board = Data0[:, 0:-1]
board = board[1].reshape((4, 4))
print(board)
board = onehot(board)
board = board.astype(np.int)

print(board)


[[0 0 0 1]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 0 0]]
[[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]]
